In [1]:
!pip3 install pandas matplotlib

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Load the data
commits_df = pd.read_csv("data_raw/commits.csv")
contributors_df = pd.read_csv("data_raw/contributors.csv")
issues_df = pd.read_csv("data_raw/issues.csv")
languages_df = pd.read_csv("data_raw/languages.csv")
pull_requests_df = pd.read_csv("data_raw/pull_requests.csv")
files_df = pd.read_csv("data_raw/files.csv")

In [4]:
print("Raw data before cleaning")
print(commits_df.isnull().sum())
print(commits_df.dtypes)
print("-----------------------")
print(contributors_df.isnull().sum())
print(contributors_df.dtypes)
print("-----------------------")
print(issues_df.isnull().sum())
print(issues_df.dtypes)
print("-----------------------")
print(languages_df.isnull().sum())
print(languages_df.dtypes)
print("-----------------------")
print(pull_requests_df.isnull().sum())
print(pull_requests_df.dtypes)
print("-----------------------")
print(files_df.isnull().sum())
print(files_df.dtypes)

Raw data before cleaning
sha               0
date              0
author            0
message           0
files_changed     0
files_commited    0
dtype: int64
sha               object
date              object
author            object
message           object
files_changed      int64
files_commited    object
dtype: object
-----------------------
login      0
commits    0
dtype: int64
login      object
commits     int64
dtype: object
-----------------------
number            0
state             0
contributor       0
title             0
comment           0
created_at        0
closed_at      1973
labels            0
dtype: int64
number          int64
state          object
contributor    object
title          object
comment         int64
created_at     object
closed_at      object
labels         object
dtype: object
-----------------------
language    0
lines       0
dtype: int64
language    object
lines        int64
dtype: object
-----------------------
number           0
state            0

In [21]:
# Làm sạch dữ liệu
commits_df = commits_df.dropna(subset=['sha', 'author'])
commits_df['message'] = commits_df['message'].fillna('')
commits_df = commits_df.drop_duplicates(subset='sha')

commits_df['date'] = pd.to_datetime(commits_df['date'])
commits_df.sort_values('date', inplace= True)
commits_df = commits_df.reset_index(drop= True)


In [32]:

# Xử lý dữ liệu

# Tính số lượng commit theo contributor
commits_contributors_df = (commits_df['author'].value_counts()).sort_values(ascending=False)

# Tính số commit theo số file thay đổi 
files_changed_commit = commits_df.explode('files_changed')['files_changed'].value_counts().reset_index()
files_changed_commit.columns = ['Files', 'Count']

# Tính số commit và contributor tổng theo tháng
commits_df['month'] = commits_df['date'].dt.tz_localize(None).dt.to_period('M')  
commits_month = commits_df.groupby('month').size()
contributors_month = commits_df.groupby('month')['author'].nunique()
growth_df = pd.DataFrame({
    'Total Commits': commits_month.cumsum(),
    'Total Contributors': contributors_month.cumsum()
})

# Tạo các edges dựa trên các contributors commit vào cùng một file
selected_commits_df = commits_df.head(400)
selected_commits_df.loc[:, 'files_commited'] = selected_commits_df['files_commited'].apply(ast.literal_eval)  # Sử dụng .loc[]

file_contributors = {}
for _, row in selected_commits_df.iterrows():
    for file in row['files_commited']:
        if (file not in file_contributors):
            file_contributors[file] = []
        file_contributors[file].append(row['author'])

edges = []
for contributors in file_contributors.values():
    for i in range(len(contributors)):
        for j in range(i + 1, len(contributors)):
            if contributors[i] != contributors[j] and contributors[i] != 'unknown' and contributors[j] != 'unknown':
                edges.append((contributors[i], contributors[j]))

edges_df = pd.DataFrame(edges, columns=['source', 'target'])
edges_df = edges_df.groupby(['source', 'target']).size().reset_index(name='weight')


In [ ]:
#Lưu dữ liệu đã qua xử lý
commits_df.to_csv('data_processed/commits.csv', index=False)
files_changed_commit.to_csv('data_processed/files_changed_commit.csv',index=False)
growth_df.to_csv('data_processed/growth.csv') #mức tăng trưởng
edges_df.to_csv('data_processed/network_graph_edges.csv', index=False) #mức đô liên quan


CLEAN CONTRIBUTOR


In [40]:
print("Contributors data before cleaning:")
print(contributors_df.isnull().sum())
print(contributors_df.dtypes)

Contributors data before cleaning:
login      0
commits    0
dtype: int64
login      object
commits     int64
dtype: object


In [36]:
print(issues_df.isnull().sum())
print("-----------------------")
print(issues_df.dtypes)

number            0
state             0
contributor       0
title             0
comment           0
created_at        0
closed_at      1973
labels            0
dtype: int64
-----------------------
number          int64
state          object
contributor    object
title          object
comment         int64
created_at     object
closed_at      object
labels         object
dtype: object


In [ ]:
# Làm sạch dữ liệu
contributors_df = contributors_df.dropna(subset=['login','commits'])
contributors_df = contributors_df.drop_duplicates(subset='login')

contributors_df['commits'] = contributors_df['commits'].astype(int) #chuyển thành số lượng 
contributors_df = contributors_df.sort_values(by='commits', ascending=False)



Xử Lý

In [ ]:
# Tính % đóng góp của mỗi contributor
total_commits = contributors_df['commits'].sum()
contributors_df['percent_contribution'] = (contributors_df['commits'] / total_commits) * 100

# Tính thời gian hoạt động của mỗi contributor
activity_periods_df = commits_df.groupby('author')['date'].agg(['min', 'max'])
activity_periods_df['activity_duration'] = (activity_periods_df['max'] - activity_periods_df['min']).dt.total_seconds() / 3600


In [43]:
# Lưu dữ liệu đã xử lí
contributors_df.to_csv('data_processed/contributors.csv', index= False)
activity_periods_df.to_csv('data_processed/activity_periods.csv')

CLEAN ISSUES

In [45]:
#CHECK
print(issues_df.describe())
print("-----------------------")
issues_df

             number closed_at
count   1962.000000         0
mean   16078.707951       NaT
min      368.000000       NaT
25%     9984.750000       NaT
50%    16158.000000       NaT
75%    22457.000000       NaT
max    27660.000000       NaT
std     7244.970794       NaN
-----------------------


,number,state,contributor,title,comment,created_at,closed_at,labels
0,27783,open,asarkar,### Describe the issue:\r\n\r\nmypy warning on...,2024-10-28 22:15:28+00:00,NaT,TYP: ``numpy.apply_along_axis`` rejects functi...,['Static typing']
1,27657,open,charris,"Strange error is showing up, see https://dev.a...",2024-10-28 19:00:41+00:00,NaT,Cargo and Rust dependency error on windows 32 ...,['unlabeled']
2,27655,open,mtelka,### Describe the issue:\r\n\r\nI'm running tes...,2024-10-28 15:43:01+00:00,NaT,BUG: `TestSpecialFloats.test_exp_exceptions` f...,['00 - Bug']
3,27777,open,chillenb,"### Describe the issue:\n\nOn x86_64, NumPy in...",2024-10-27 23:59:14+00:00,NaT,int64 is not supported by the standard library...,"['01 - Enhancement', '57 - Close?']"
4,27639,open,xuwinnie,### Proposed new feature or change:\n\nit woul...,2024-10-25 03:06:48+00:00,NaT,ENH: show an warning when np.maximum received ...,"['07 - Deprecation', 'sprintable - C']"
...,...,...,...,...,...,...,...,...
1957,630,open,numpy-gitbot,_Original ticket http://projects.scipy.org/num...,2012-10-19 15:10:24+00:00,NaT,Add step parameter to linspace (or endpoint pa...,"['01 - Enhancement', 'component: numpy._core']"
1958,626,open,numpy-gitbot,_Original ticket http://projects.scipy.org/num...,2012-10-19 15:10:18+00:00,NaT,npyio.py: Make .squeeze() an option for genfro...,"['00 - Bug', 'component: numpy.lib']"
1959,619,open,numpy-gitbot,_Original ticket http://projects.scipy.org/num...,2012-10-19 15:10:06+00:00,NaT,BLAS matrix product (dot) never used for ndim ...,"['01 - Enhancement', 'component: numpy._core']"
1960,389,open,RONNCC,I have a file (exactly);\n\n```\n# Data from '...,2012-08-18 21:22:46+00:00,NaT,GenFromText should be able to read field names...,"['00 - Bug', 'component: numpy.lib']"


In [6]:
# Làm sạch dữ liệu
issues_df = issues_df.dropna(subset=['contributor' ,'title', 'created_at'])  # Loại bỏ giá trị trống
issues_df['created_at'] = pd.to_datetime(issues_df['created_at'])  # Chuyển đổi kiểu dữ liệu
issues_df['closed_at'] = pd.to_datetime(issues_df['closed_at'])  # Chuyển đổi kiểu dữ liệu
issues_df['created_at'] = issues_df['created_at'].dt.tz_localize(None)
issues_df['closed_at'] = issues_df['closed_at'].dt.tz_localize(None)


print(issues_df['closed_at'].isna().sum())#kiểm tra số lượng issues đã đóng
issues_df['is_closed'] = issues_df['closed_at'].notna()
issues_df = issues_df[issues_df['closed_at'] >= issues_df['created_at']]




1973


In [ ]:

#Xử lý 
# Tính tổng số issues
total_issues = len(issues_df)
open_issues = issues_df[issues_df['state'] == 'open'].shape[0]
closed_issues = issues_df[issues_df['state'] == 'closed'].shape[0]
print(closed_issues)

# Tính thời gian giải quyết các issues đã đóng
closed_issues_df = issues_df[issues_df['closed_at'].notna()]
closed_issues_df['resolution_time'] = (closed_issues_df['closed_at'] - closed_issues_df['created_at']).dt.total_seconds() / 3600

# Tính số issues theo label
label_counts_df = issues_df.explode('labels')['labels'].value_counts().reset_index()
label_counts_df.columns = ['label', 'count']

# Tính số issues đã đóng theo contributor
issues_contributor_df = issues_df[issues_df['state'] == 'closed']['contributor'].value_counts().reset_index()
issues_contributor_df.columns = ['contributor', 'closed_issues_count']

# Tính số issues theo người đóng góp
issues_per_contributor_df = issues_df['contributor'].value_counts().reset_index()
issues_per_contributor_df.columns = ['contributor', 'issues_count']

# Tạo bảng tổng hợp thông tin
summary_df = pd.DataFrame({
    'Total Issues': [total_issues],
    'Open Issues': [open_issues],
    'Closed Issues': [closed_issues],
    'Average Resolution Time (hrs)': [closed_issues_df['resolution_time'].mean() if not closed_issues_df.empty else 0]
})
#Issues mở quá lâu
stale_issues_df = issues_df[(issues_df['state'] == 'open') & 
                            ((pd.to_datetime("today") - issues_df['created_at']).dt.days > 90)]

contributor_resolution_time = closed_issues_df.groupby('contributor')['resolution_time'].mean().reset_index()
contributor_resolution_time.columns = ['contributor', 'average_resolution_time']


10906


In [8]:
#Lưu
issues_df.to_csv('data_processed/issues_cleaned.csv', index=False)

# Lưu tổng hợp thông tin về issues vào file CSV
summary_df.to_csv("data_processed/summary_issues.csv", index=False)

# Lưu số lượng issues theo label vào file CSV
label_counts_df.to_csv("data_processed/label_counts.csv", index=False)

# Lưu số lượng issues đã đóng theo contributor vào file CSV
#issues_contributor_df.to_csv("data_processed/closed_issues_per_contributor.csv", index=False)

# # Phân tích tần suất từ
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(issues_df['filtered_content'])
# word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


In [ ]:
#Lưu
issues_df.to_csv('data_cleaned/issues_cleaned.csv', index=False)
label_counts_df.to_csv('data_cleaned/label_counts.csv', index=False)
issues_contributor_df.to_csv('data_cleaned/issues_contributor_summary.csv', index=False)
summary_df.to_csv('data_cleaned/issues_summary.csv', index=False)
issues_per_contributor_df.to_csv('data_cleaned/issues_per_contributor.csv', index=False)



CLEAN PULL_REQUEST

In [9]:
# Kiểm tra chung
pull_requests_df

,number,state,created_at,merged_at,closed_at,title
0,27669,closed,2024-10-30 00:36:04+00:00,2024-10-30 03:59:58+00:00,2024-10-30 03:59:58+00:00,ENH: fix wasm32 runtime type error in numpy._core
1,27668,closed,2024-10-30 00:31:53+00:00,2024-10-30 03:59:11+00:00,2024-10-30 03:59:11+00:00,BLD: Do not set __STDC_VERSION__ to zero durin...
2,27667,open,2024-10-29 21:56:48+00:00,NaN,NaN,TYP: Allow returning non-array-likes from the ...
3,27666,open,2024-10-29 18:59:06+00:00,NaN,NaN,BUG: Fix a reference count leak in npy_find_de...
4,27665,open,2024-10-29 17:31:53+00:00,NaN,NaN,ENH: Re-enable VXE from build targets for sin/...
...,...,...,...,...,...,...
14776,5,closed,2010-10-19 19:42:36+00:00,NaN,2010-11-20 00:28:55+00:00,Fix structured compare
14777,4,closed,2010-10-18 19:04:33+00:00,NaN,2010-10-21 15:44:32+00:00,Whitespace cleanup
14778,3,closed,2010-10-17 04:49:24+00:00,2010-10-17 05:07:48+00:00,2010-10-17 05:07:48+00:00,Poly1d failure
14779,2,closed,2010-10-16 14:32:13+00:00,2010-10-17 06:04:05+00:00,2010-10-17 06:04:05+00:00,Deprecation docs


In [ ]:
# Làm sạch dữ liệu

pull_requests_df = pull_requests_df.dropna(subset=['number'])
pull_requests_df = pull_requests_df.drop_duplicates(subset='number')
pull_requests_df['title'] = pull_requests_df['title'].str.lower()

In [ ]:
#Xủ lý
# Lọc các pull request đã được merge
merged_pull_requests_df = pull_requests_df[pull_requests_df['merged_at'].notna()].copy()  # Sử dụng copy() để tạo bản sao

# Tính thời gian để merge
merged_pull_requests_df['time_to_merge'] = (
    pd.to_datetime(merged_pull_requests_df['merged_at']) - 
    pd.to_datetime(merged_pull_requests_df['created_at'])
).dt.total_seconds() / 3600

# Lọc các pull request chưa được merge nhưng đã đóng
unmerged_pull_requests_df = pull_requests_df[
    (pull_requests_df['merged_at'].isna()) & 
    (pull_requests_df['closed_at'].notna())
].copy()  # Sử dụng copy() để tạo một bản sao độc lập

# Xóa cột 'merged_at' khỏi unmerged_pull_requests_df
unmerged_pull_requests_df = unmerged_pull_requests_df.drop('merged_at', axis=1)

In [65]:
# Lưu dữ liệu đã xử lí

pull_requests_df.to_csv('data_processed/pull_requests.csv', index=False)
merged_pull_requests_df.to_csv('data_processed/merged_pull_requests.csv', index=False)
unmerged_pull_requests_df.to_csv('data_processed/unmerged_pull_requests.csv', index=False)

CLEAN LANGUAGES


In [66]:
# Kiểm tra chung
languages_df

,language,lines
0,Python,11076549
1,C,6392582
2,C++,473962
3,Cython,174675
4,Meson,95304
5,Fortran,37029
6,Shell,17823
7,sed,5706
8,Smarty,4129
9,Starlark,1842


In [67]:
# Xử lí dữ liệu

# Loại bỏ các dòng có giá trị trống trong cột 'language'
languages_df = languages_df.dropna(subset=['language'])

# Loại bỏ các dòng trùng lặp trong cột 'language' và giữ lại lần xuất hiện đầu tiên
languages_df = languages_df.drop_duplicates(subset=['language'], keep='first')



In [68]:
# Lưu dữ liệu đã xử lí
languages_df.to_csv('data_cleaned/languages.csv')

Đã xử lý file languages.csv và lưu kết quả vào languages.csv.


CLEAN FILE

In [69]:
# Kiểm tra chung
#dữ liệu khi chưa xử lý
files_df

,path,size
0,.cirrus.star,1842
1,.clang-format,1133
2,.codecov.yml,230
3,.coveragerc,75
4,.ctags.d,19
...,...,...
2216,doc/source/f2py/code/advanced/boilerplating/sr...,984
2217,doc/source/f2py/code/advanced/boilerplating/sr...,328
2218,doc/source/f2py/code/advanced/boilerplating/sr...,807
2219,doc/source/f2py/code/advanced/boilerplating/sr...,576


In [71]:
# Làm sạch dữ liệu
files_df = files_df.dropna(subset=['path', 'size'])
files_df = files_df.drop_duplicates(subset='path')

files_df['path'] = files_df['path'].str.lower()
files_df['extension'] = files_df['path'].apply(lambda x: x.split('.')[-1] if '.' in x else 'no_extension')

In [72]:
# Xử lí dữ liệu

# Tính số files theo loại
file_extension_counts_df = files_df['extension'].value_counts()
count_threshold = 0.01 * files_df.shape[0]
file_extension_counts_revised_df = file_extension_counts_df.copy()
file_extension_counts_revised_df[file_extension_counts_df < count_threshold] = 0
file_extension_counts_revised_df['other'] = file_extension_counts_df[file_extension_counts_df < count_threshold].sum()
file_extension_counts_revised_df = file_extension_counts_revised_df[file_extension_counts_revised_df > 0]

# Tính kích thước files theo loại
file_extension_sizes_df = files_df.groupby('extension')['size'].sum()
size_threshold = 0.01 * file_extension_sizes_df.sum()
file_extension_sizes_revised_df = file_extension_sizes_df.copy()
file_extension_sizes_revised_df[file_extension_sizes_df < size_threshold] = 0
file_extension_sizes_revised_df['other'] = file_extension_sizes_df[file_extension_sizes_df < size_threshold].sum()
file_extension_sizes_revised_df = file_extension_sizes_revised_df[file_extension_sizes_revised_df > 0]


In [74]:
# Lưu dữ liệu đã xử lí
files_df.to_csv('data_processed/files.csv', index= False)
file_extension_counts_revised_df.to_csv('data_processed/file_extension_counts.csv')
file_extension_sizes_revised_df.to_csv('data_processed/file_extension_sizes.csv')